This notebook is used to do some feature engineering.
<br />
E.g., calculate the incumbetn party: the incumbent party is the party that won the previous election in that constituency.
<br />

### **PLEASE NOTE!**
For 2010 we need to get 2005 data, with the complications that in 2005 we do not have a correct ID, so we need to use the name
From 2015 onwards, we can use the ID of the consitutency


In [2]:
import pandas as pd
from google.cloud import storage


In [3]:
data = pd.read_csv('/Users/andreabrumana/code/willgreen93/UK_election/raw_data/cleaned_election_results.csv')
data.head()


,year,constituency_id,constituency,country,electorate,total_votes,tornout,conservative_votes,conservative_vote_share,labour_votes,labour_vote_share,liberal_democrats_votes,liberal_democrats_vote_share,main_parties_votes,main_parties_vote_share,other_parties_votes,other_parties_vote_share
0,2001,1,BEDFORD,England,67762,40579,0.598846,13297.0,0.327682,19454.0,0.479411,6425.0,0.158333,39176.0,0.965425,1403.0,0.034575
1,2001,2,LUTON NORTH,England,67554,39126,0.579181,12210.0,0.312069,22187.0,0.567065,3795.0,0.096994,38192.0,0.976128,934.0,0.023872
2,2001,3,LUTON SOUTH,England,71439,39351,0.550834,11586.0,0.294427,21719.0,0.551930,4292.0,0.109070,37597.0,0.955427,1754.0,0.044573
3,2001,4,MID BEDFORDSHIRE,England,70794,46638,0.658785,22109.0,0.474055,14043.0,0.301106,9205.0,0.197371,45357.0,0.972533,1281.0,0.027467
4,2001,5,NORTH EAST BEDFORDSHIRE,England,69877,45246,0.647509,22586.0,0.499182,14009.0,0.309619,7409.0,0.163749,44004.0,0.972550,1242.0,0.027450


In [4]:
data['winning_party'] = data[['conservative_vote_share', 'labour_vote_share', 'liberal_democrats_vote_share', 'other_parties_vote_share']].idxmax(axis=1).str.replace('_vote_share', '')


In [5]:
data['incumbent_party'] = data.groupby('constituency_id')['winning_party'].shift(1)


In [6]:
data['conservative_vote_share_previous'] = data.groupby('constituency_id')['conservative_vote_share'].shift(1)
data['labour_vote_share_previous'] = data.groupby('constituency_id')['labour_vote_share'].shift(1)
data['liberal_democrats_vote_share_previous'] = data.groupby('constituency_id')['liberal_democrats_vote_share'].shift(1)
data['other_parties_vote_share_previous'] = data.groupby('constituency_id')['other_parties_vote_share'].shift(1)


In [7]:
data['winning_party_changed'] = data['winning_party'] != data['incumbent_party']


In [8]:
final_data = data[data['year']>2005].reset_index(drop=True)
final_data


,year,constituency_id,constituency,country,electorate,total_votes,tornout,conservative_votes,conservative_vote_share,labour_votes,...,main_parties_vote_share,other_parties_votes,other_parties_vote_share,winning_party,incumbent_party,conservative_vote_share_previous,labour_vote_share_previous,liberal_democrats_vote_share_previous,other_parties_vote_share_previous,winning_party_changed
0,2010,E14000530,ALDERSHOT,England,71465,45384,0.635052,21203.0,0.467191,5489.0,...,0.932245,3075.0,0.067755,conservative,conservative,0.427328,0.205542,0.316529,0.050601,False
1,2010,E14000531,ALDRIDGE-BROWNHILLS,England,58909,38634,0.655825,22913.0,0.593079,7647.0,...,0.967878,1241.0,0.032122,conservative,conservative,0.473860,0.334639,0.122914,0.068586,False
2,2010,E14000532,ALTRINCHAM AND SALE WEST,England,72208,49393,0.684038,24176.0,0.489462,11073.0,...,0.968356,1563.0,0.031644,conservative,conservative,0.464207,0.302640,0.216543,0.016610,False
3,2010,E14000533,AMBER VALLEY,England,70171,45958,0.654943,17746.0,0.386135,17210.0,...,0.905000,4366.0,0.095000,conservative,labour,0.344327,0.455635,0.131354,0.068684,True
4,2010,E14000534,ARUNDEL AND SOUTH DOWNS,England,76835,55982,0.728600,32333.0,0.577561,4835.0,...,0.943339,3172.0,0.056661,conservative,conservative,0.498128,0.170698,0.270537,0.060636,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,2019,E14001059,WYTHENSHAWE AND SALE EAST,England,76313,44759,0.586519,13459.0,0.300699,23855.0,...,0.903170,4334.0,0.096830,labour,labour,0.296231,0.622192,0.032805,0.048772,False
2596,2019,E14001060,YEOVIL,England,82468,59260,0.718582,34588.0,0.583665,3761.0,...,0.957746,2504.0,0.042254,conservative,conservative,0.544896,0.124874,0.297051,0.033180,False
2597,2019,W07000041,YNYS MON,Wales,51925,36552,0.703938,12959.0,0.354536,10991.0,...,0.655231,12602.0,0.344769,conservative,labour,0.277892,0.418631,0.012819,0.290658,True
2598,2019,E14001061,YORK CENTRAL,England,74899,49505,0.660957,13767.0,0.278093,27312.0,...,0.913605,4277.0,0.086395,labour,labour,0.301744,0.651635,0.046621,0.000000,False


In [9]:
final_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600 entries, 0 to 2599
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   year                                   2600 non-null   int64  
 1   constituency_id                        2600 non-null   object 
 2   constituency                           2600 non-null   object 
 3   country                                2600 non-null   object 
 4   electorate                             2600 non-null   int64  
 5   total_votes                            2600 non-null   int64  
 6   tornout                                2600 non-null   float64
 7   conservative_votes                     2600 non-null   float64
 8   conservative_vote_share                2600 non-null   float64
 9   labour_votes                           2600 non-null   float64
 10  labour_vote_share                      2600 non-null   float64
 11  libe

In [10]:
final_data[final_data['labour_vote_share_previous'].isnull()]


,year,constituency_id,constituency,country,electorate,total_votes,tornout,conservative_votes,conservative_vote_share,labour_votes,...,main_parties_vote_share,other_parties_votes,other_parties_vote_share,winning_party,incumbent_party,conservative_vote_share_previous,labour_vote_share_previous,liberal_democrats_vote_share_previous,other_parties_vote_share_previous,winning_party_changed
73,2010,E14000603,BROADLAND,England,72445,52676,0.727117,24338.0,0.462032,7287.0,...,0.923969,4005.0,0.076031,conservative,NaN,NaN,NaN,NaN,NaN,True
93,2010,E14000623,CENTRAL DEVON,England,71203,53873,0.756611,27737.0,0.514859,3715.0,...,0.927348,3914.0,0.072652,conservative,NaN,NaN,NaN,NaN,NaN,True
99,2010,E14000629,CHELSEA AND FULHAM,England,66257,39856,0.601536,24093.0,0.604501,7371.0,...,0.951852,1919.0,0.048148,conservative,NaN,NaN,NaN,NaN,NaN,True
105,2010,E14000635,CHIPPENHAM,England,72106,52385,0.726500,21500.0,0.410423,3620.0,...,0.937100,3295.0,0.062900,liberal_democrats,NaN,NaN,NaN,NaN,NaN,True
172,2010,E14000702,FILTON AND BRADLEY STOKE,England,69003,48301,0.699984,19686.0,0.407569,12772.0,...,0.924515,3646.0,0.075485,conservative,NaN,NaN,NaN,NaN,NaN,True
237,2010,E14000767,KENILWORTH AND SOUTHAM,England,64362,48431,0.752478,25945.0,0.535711,6949.0,...,0.955731,2144.0,0.044269,conservative,NaN,NaN,NaN,NaN,NaN,True
281,2010,E14000811,MEON VALLEY,England,70488,51238,0.726904,28818.0,0.562434,3266.0,...,0.951969,2461.0,0.048031,conservative,NaN,NaN,NaN,NaN,NaN,True
284,2010,E14000814,MID DERBYSHIRE,England,66074,47342,0.716500,22877.0,0.483228,11585.0,...,0.933062,3169.0,0.066938,conservative,NaN,NaN,NaN,NaN,NaN,True
403,2010,E14000933,SOUTH BASILDON AND EAST THURROCK,England,71874,44735,0.622409,19624.0,0.438672,13852.0,...,0.881927,5282.0,0.118073,conservative,NaN,NaN,NaN,NaN,NaN,True
412,2010,E14000942,SOUTH NORTHAMPTONSHIRE,England,82033,59890,0.730072,33081.0,0.552363,10380.0,...,0.936116,3826.0,0.063884,conservative,NaN,NaN,NaN,NaN,NaN,True


In [23]:
# Save as csv
final_data.to_csv('/Users/andreabrumana/code/willgreen93/UK_election/raw_data/cleaned_elections_data_with_incumbent.csv', index=False)


In [14]:
client = storage.Client(project="uk-election-406413")
bucket = client.bucket("ukelectiondata")
cleaned_election_results_with_incumbent_blob = bucket.blob("cleaned_elections_data_with_incumbent.csv")
cleaned_election_results_with_incumbent_blob.upload_from_filename("/Users/andreabrumana/code/willgreen93/UK_election/raw_data/cleaned_elections_data_with_incumbent.csv")
